# Урок 12: 2025-2026 SOTA Advancements

В этом уроке мы разберем «передний край» технологий сжатия, которые стали стандартом в 2025-2026 годах. Мы переходим от простого квантования готовых моделей к обучению изначально сжатых архитектур и динамическому управлению контекстом.

## 1. Математическая теория

### 1.1. BitNet b1.58 (Ternary Quantization)
Главный прорыв 2025 года — отказ от классических битов. Вместо того чтобы квантовать FP16 в INT4, **BitNet** предлагает использовать веса $\in \{-1, 0, 1\}$. 
Математически это $\log_2(3) \approx 1.58$ бит. 
Преимущество: Умножение матрицы на вектор заменяется простым сложением и вычитанием, что дает колоссальную экономию энергии на кристалле.

### 1.2. FOEM (First-Order Error Matters) [2025]
Классический GPTQ минимизирует ошибку весов. FOEM идет дальше: он использует градиент первого порядка (якобиан) функции потерь, чтобы понять, какие изменения веса сильнее всего бьют по качеству ответа. Это позволяет достичь качества FP16 при сжатии до 3 бит.

### 1.3. SnapKV / Context Pruning (2026)
Для моделей с контекстом 1M+ токенов даже 2-битный KV-кэш становится слишком тяжелым. 
**SnapKV** анализирует паттерны внимания и обнаруживает, что для генерации следующего токена реально нужны менее 5% предыдущих ключей. Остальные 95% кэша динамически «выбрасываются» (evicted) без потери смысла.

---

In [ ]:
import torch
import torch.nn as nn

def bitnet_quantize_raw(w):
    """ Реализация логики BitNet b1.58 ({-1, 0, 1}) """
    scale = w.abs().mean()
    # Квантуем в {-1, 0, 1}
    w_q = torch.round(w / (scale + 1e-6)).clamp(-1, 1)
    return w_q, scale

class BitLinear(nn.Linear):
    def forward(self, x):
        # В BitNet активации квантуются в INT8, а веса в 1.58 бит
        w_q, scale = bitnet_quantize_raw(self.weight)
        # Вместо умножения здесь могла бы быть оптимизированная сумма
        return torch.nn.functional.linear(x, w_q * scale, self.bias)

print("nanoGPT Track: Реализован прототип BitLinear (1.58-bit).")

In [ ]:
def snapkv_prune_cache_raw(key_cache, attn_scores, keep_ratio=0.1):
    """ Симуляция SnapKV: оставляем только самые важные ключи в кеше """
    # attn_scores: (B, H, T, T) - усредненные веса внимания
    importance = attn_scores.mean(dim=(0, 1, 2)) # Важность каждого токена
    k = int(len(importance) * keep_ratio)
    
    top_indices = torch.topk(importance, k).indices
    pruned_cache = key_cache[:, :, top_indices, :]
    return pruned_cache

print("nanoGPT Track: Реализована логика Context Pruning (SnapKV).")

## 2. Промышленная реализация: BitNet-a & Speculative Decoding
В 2026 году Microsoft и Hugging Face выпустили библиотеки для работы с нативными 1-битными весами. 
Одновременно с этим, **Speculative Decoding** (через библиотеки `vLLM` или `TensorRT-LLM`) стал обязательным: сжатая модель (Draft) предсказывает 5-10 токенов вперед, а тяжелая модель (Target) проверяет их за один проход.

In [ ]:
"""
# Пример Speculative Decoding в vLLM [2026]
from vllm import LLM, SamplingParams

llm = LLM(
    model="deepseek-67b",
    speculative_model="deepseek-7b-quantized", # Наша сжатая модель
    num_speculative_tokens=5
)
"""
print("Llama Track: Speculative Decoding ускоряет инференс в 3 раза без потери точности.")